<p align="center">
    <img src="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/raw/main/images/triangles_fig.png" height="350" alt="tracking image">
</p>

# Annotating triangular crystallites
---

**Note:** This is an advanced example!

In this notebook, we will explore some of the interactive capabilities of Napari. We will use the `napari-matplotlib` plugin to integrate a histogram plot into the viewer and update it as the user draws polygons in a `Shapes` layer. The histogram represents the tilt angle of the crystallites (triangles) with respect to the horizontal axis of the image.

### Acknowledgements

We kindly acknowledge the [Laboratory of Advanced Separations](https://www.epfl.ch/labs/las/) in EPFL for providing the data for this notebook!

### Setup

Check that you have all the necessary packages installed, including `napari`, the `napari-matplotlib` plugin, and `seaborn`. If not, you can use the `!` symbol to install them directly from the Jupyter notebook (otherwise, you can use your terminal).

In [10]:
import napari
from napari_matplotlib.base import NapariMPLWidget
from napari_matplotlib.util import Interval

### Get the data

The image we'll use in this tutorial is available for download on [Zenodo](https://zenodo.org/record/8099852) (`crystallites.tif`).

In the cell below, we use a Python package called [pooch](https://pypi.org/project/pooch/) to automatically download the image from Zenodo into the **data** folder of this repository.

In [11]:
import pooch
from pathlib import Path

data_path = Path('.').resolve().parent / 'data'
fname = 'crystallites.tif'

pooch.retrieve(
    url="https://zenodo.org/record/8099852/files/crystallites.tif",
    known_hash="md5:18d619a8f70114f2e5437e4713e45166",
    path=data_path,
    fname=fname,
    progressbar=True,
)

print(f'Downloaded image {fname} into: {data_path}')

Downloaded image crystallites.tif into: /home/wittwer/code/eias-2023-visualization-workshop/data


### Read the image

We use the `imread` function from Scikit-image to read our TIF image.

In [12]:
from skimage.io import imread

image = imread(data_path / 'crystallites.tif')

print(f'Loaded image in an array of shape: {image.shape} and data type {image.dtype}')
print(f'Intensity range: [{image.min()} - {image.max()}]')

Loaded image in an array of shape: (487, 849) and data type uint8
Intensity range: [90 - 255]


### Load the image into Napari

Let's open a viewer and load our image to have a look at it.

In [13]:
viewer = napari.Viewer()
viewer.add_image(image)

/home/wittwer/miniconda3/envs/eias/lib/python3.9/site-packages/napari/plugins/_plugin_manager.py:555: UserWarning: Plugin 'napari_skimage_regionprops2' has already registered a function widget 'duplicate current frame' which has now been overwritten
  warn(message=warn_message)


<Image layer 'image' at 0x7fdeb045bdf0>

### Set up custom UI interactions

First, we create a class `AngleHistogramWidget` that inherits from the base class `NapariMPLWidget` from the `napari-matplotlib` plugin to define the kind of plot to draw. The code can be templated and reused to display any kind of plot compatible with `matplotlib`.

Here, for example, we display a [Seaborn histplot](https://seaborn.pydata.org/generated/seaborn.histplot.html) in the interface.

In [14]:
import numpy as np
import seaborn as sns
import napari.layers

class AngleHistogramWidget(NapariMPLWidget):
    """
    Displays a histogram of the 'angle' property of the currently selected Napari `Shapes` layer.
    """
    n_layers_input = Interval(1, 1)
    input_layer_types = (napari.layers.Shapes,)

    def __init__(self, napari_viewer: napari.viewer.Viewer):
        super().__init__(napari_viewer)
        self.axes = self.canvas.figure.subplots()
        self.axes.set_xlabel('Tilt angle (deg.)')
        self._update_layers(None)

    def draw(self) -> None:
        layer = self.layers[0]
        if not 'angle' in list(layer.properties.keys()):
            return
        
        # The seaborn histplot looks for data in the 'angle' property of the layer.
        sns.histplot(layer.properties['angle'], bins=np.linspace(-70, 70, 50), ax=self.axes)
    
    def clear(self) -> None:
        self.axes.clear()


# Create and dock the Histogram widget element
histo_widget = AngleHistogramWidget(viewer)
viewer.window.add_dock_widget(histo_widget, name='Orientation distribution');

Next, we define a callback function `on_set_data` that we connect to the event `set_data` of the Napari `Shapes` layer used for annotating the crystallites. In this way, we can update the text to display in the layer dynamically as the user draws new polygons. Similarly, we connect the `mouse_double_click` callback of the layer to the method `_draw` of the `AngleHistogramWidget` so that the histogram gets updated when the user finishes drawing a polygon.

If you're interested in setting up custom interactions for your own project, you can check the [Events reference](https://napari.org/stable/guides/events_reference.html) and [this tutorial](https://napari.org/dev/howtos/connecting_events.html).

In [15]:
import pandas as pd
from napari.utils.events import Event

def on_set_data(event: Event):
    """Called when the data in the `Shapes` annotation layer changes."""
    shapes_layer = event.source
    shapes_data = shapes_layer.data

    if len(shapes_data) == 0:
        return

    angles = []
    for polygon in shapes_data:
        triangle = pd.DataFrame.from_dict({'y': polygon[:, 0], 'x': polygon[:, 1]})
        if len(triangle) < 3:
            tilt_angle_degrees = 0.0
        else:
            # Sort the corners by Y coordinate
            triangle.sort_values(by='y', inplace=True)
            
            # Select the two corner points opposite to the lowest corner, sort them by X coordinate
            points_opposite = triangle.iloc[1:].sort_values(by='x')

            # Compute the deltas (dX, dY)
            deltas = points_opposite[['x', 'y']].diff().iloc[1]

            # Avoid dividing by zero if dX is zero
            if deltas['x'] == 0:
                tilt_angle_degrees = 0.0
            else:
                tilt_angle_radians = np.arctan(deltas['y'] / deltas['x'])
                tilt_angle_degrees = np.degrees(tilt_angle_radians)
                
        angles.append(tilt_angle_degrees)

    shapes_layer.properties = {'angle': angles}
    shapes_layer.text={
        'string': '{angle:.2f} deg.',
        'size': 16,
        'color': 'black',
    }
    shapes_layer.face_color='angle'


# Add a `Shapes` layer in which to draw the polygons
shapes_layer = viewer.add_shapes(data=None, name='Annotations', shape_type='polygon')

# Call `on_set_data` when the user sets data into the `Shapes` layer
shapes_layer.events.set_data.connect(on_set_data)

# Update the histogram on double-click (to finish drawing a polygon)
shapes_layer.mouse_double_click_callbacks.append(lambda layer, event: histo_widget._draw())

### Usage

- Select the `Annotations` layer
- Draw polygons on the triangular crystallites
- `double left-click` to finish drawing a polygon and move to the next!

<p align="center">
    <a href="https://gitlab.epfl.ch/center-for-imaging/eias-2023-visualization-workshop/-/blob/main/examples/README.md">🔙 Back to case studies</a>
</p>
